# Water Level Detection - Image Gatherer

This notebook captures training images from the OpenMV camera for water level detection.

## Purpose
- Connect to OpenMV camera via serial
- Capture multiple images of the water tube at different levels
- Save images to folder for later training

## Hardware Setup
1. Connect OpenMV H7 camera via USB
2. Position camera to view the water tube
3. Ensure consistent lighting
4. Note the COM port of the camera

## Detect Available Ports

First, let's check which COM ports are available on your system to identify the OpenMV camera.

In [34]:
import serial.tools.list_ports

# List all available serial ports
ports = serial.tools.list_ports.comports()

print("Available Serial Ports:")
print("=" * 60)

if not ports:
    print("No serial ports found!")
else:
    for port in ports:
        print(f"\nPort: {port.device}")
        print(f"  Description: {port.description}")
        print(f"  Hardware ID: {port.hwid}")
        
        # Highlight potential OpenMV camera
        if "USB" in port.description.upper() or "OpenMV" in port.description.upper():
            print("  👉 This might be your OpenMV camera!")

print("\n" + "=" * 60)
print("\n💡 Tip: Look for ports with 'USB Serial' or similar descriptions")
print("   Update the PORT variable in the next cell with the correct port.")

Available Serial Ports:

Port: COM10
  Description: USB Serial Device (COM10)
  Hardware ID: USB VID:PID=37C5:124A SER=337F366B3131 LOCATION=1-1:x.1
  👉 This might be your OpenMV camera!

Port: COM3
  Description: Standard Serial over Bluetooth link (COM3)
  Hardware ID: BTHENUM\{00001101-0000-1000-8000-00805F9B34FB}_VID&000105D6_PID&000A\7&2CE0C528&0&41BB00444129_C00000000

Port: COM5
  Description: Standard Serial over Bluetooth link (COM5)
  Hardware ID: BTHENUM\{00001101-0000-1000-8000-00805F9B34FB}_VID&0001009E_PID&4068\7&2CE0C528&0&BC87FAD422E8_C00000000

Port: COM4
  Description: Standard Serial over Bluetooth link (COM4)
  Hardware ID: BTHENUM\{00001101-0000-1000-8000-00805F9B34FB}_LOCALMFG&0000\7&2CE0C528&0&000000000000_00000002

Port: COM8
  Description: Standard Serial over Bluetooth link (COM8)
  Hardware ID: BTHENUM\{00001101-0000-1000-8000-00805F9B34FB}_LOCALMFG&0000\7&2CE0C528&0&000000000000_00000006

Port: COM6
  Description: Standard Serial over Bluetooth link (COM6)
 

## Configuration & Connection

Set up serial connection to OpenMV camera and define image capture function.

In [35]:
import serial
import struct
import time
from pathlib import Path

# Configuration
PORT = "COM10"  # Change to your OpenMV port (see above for detected ports)
BAUD = 115200
SAVE_DIR = Path("data/raw_images")

# Create directory if it doesn't exist
SAVE_DIR.mkdir(parents=True, exist_ok=True)

print(f"✓ Configuration loaded")
print(f"  Port: {PORT}")
print(f"  Baud rate: {BAUD}")
print(f"  Save directory: {SAVE_DIR.absolute()}")

✓ Configuration loaded
  Port: COM10
  Baud rate: 115200
  Save directory: d:\Code\liquid-measure\data\raw_images


### Connect to Camera

In [36]:
# Connect to OpenMV camera
try:
    ser = serial.Serial(PORT, BAUD, timeout=2)
    time.sleep(2)  # Wait for connection to stabilize
    print(f"✓ Successfully connected to OpenMV on {PORT}")
except Exception as e:
    print(f"✗ Failed to connect: {e}")
    print("  Make sure the camera is plugged in and the PORT is correct.")

✓ Successfully connected to OpenMV on COM10


### Define Capture Function

In [37]:
def capture_image(idx):
    """
    Capture a single image from OpenMV camera.
    
    Args:
        idx: Image index for filename
        
    Returns:
        Path to saved image file
    """
    # Send capture command to camera
    ser.write(b"CAPTURE\n")
    
    # Wait for acknowledgment
    ack = ser.readline().decode().strip()
    if ack != "OK":
        raise RuntimeError("Camera did not respond with OK")
    
    # Read image size (4 bytes, little-endian unsigned int)
    size = struct.unpack("<I", ser.read(4))[0]
    
    # Read image data
    img_bytes = ser.read(size)
    
    # Save to file
    filename = SAVE_DIR / f"img_{idx:04d}.jpg"
    with open(filename, "wb") as f:
        f.write(img_bytes)
    
    print(f"✓ Saved: {filename}")
    return filename

print("✓ Capture function defined")

✓ Capture function defined


### Test Camera Connection (Optional)

Run this cell to capture a single test image and verify everything is working.

In [38]:
# Capture a test image
try:
    test_img = capture_image(0)
    print(f"\n✓ Test successful! Camera is working correctly.")
    print(f"  Test image saved to: {test_img}")
except Exception as e:
    print(f"✗ Test failed: {e}")
    print("  Check camera connection and OpenMV script.")

✓ Saved: data\raw_images\img_0000.jpg

✓ Test successful! Camera is working correctly.
  Test image saved to: data\raw_images\img_0000.jpg


## Capture Training Images

Capture multiple images for training the water level detection model.

**Instructions:**
1. Adjust `N` to set how many images to capture
2. Manually change water levels between captures if needed
3. Run the cell to start capturing
4. Each image will be saved with a sequential number

In [39]:
# Number of images to capture
N = 2

print(f"Starting capture of {N} images...\n")

for i in range(N):
    try:
        capture_image(i)
        time.sleep(0.5)  # Delay between captures
    except Exception as e:
        print(f"✗ Error capturing image {i}: {e}")
        break

print(f"\n✓ Successfully captured {N} images!")
print(f"✓ Images saved to: {SAVE_DIR.absolute()}")

Starting capture of 2 images...

✓ Saved: data\raw_images\img_0000.jpg
✓ Saved: data\raw_images\img_0001.jpg

✓ Successfully captured 2 images!
✓ Images saved to: d:\Code\liquid-measure\data\raw_images


## Cleanup

Close the serial connection when finished.

In [40]:
ser.close()
print("✓ Serial connection closed")
print("✓ Ready for next step: image labeling and training")

✓ Serial connection closed
✓ Ready for next step: image labeling and training


## Next Steps

After capturing images:

1. **Review captured images** in the `data/raw_images/` folder
2. **Label images** with corresponding water levels (create CSV or JSON file)
3. **Train the model** using the `training.ipynb` notebook
4. **Test the model** with new images using `imageRun.ipynb`